In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd

# Always make it pretty.
plt.style.use('ggplot')

In [2]:
df1 = pd.read_csv("data/accidents_2005_to_2007.csv")
df2 = pd.read_csv("data/accidents_2009_to_2011.csv")
df3 = pd.read_csv("data/accidents_2012_to_2014.csv")
traffic = pd.read_csv("data/ukTrafficAADF.csv")

df12 = pd.concat([df1,df2])
df = pd.concat([df12,df3])

df = df.drop_duplicates(subset=['Accident_Index','Date','LSOA_of_Accident_Location','Time','Longitude','Latitude'], keep='first')

df = df[df.Speed_limit !=15]

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
del df['Police_Force']
del df['Local_Authority_(Highway)']
del df['Local_Authority_(District)']
del df['Pedestrian_Crossing-Human_Control']
del df['Pedestrian_Crossing-Physical_Facilities']
del df['Did_Police_Officer_Attend_Scene_of_Accident']

In [4]:
severity_totals = df.groupby('Accident_Severity').size().values
#total is 1,469,963 number of accidents 1469963
severity_mus = severity_totals / severity_totals.sum()
severity_mus

array([0.01295348, 0.13531997, 0.85172655])

In [5]:
actuals_severity_slimits = np.transpose(np.reshape(df.groupby(['Accident_Severity','Speed_limit']).count()['Accident_Index'].values,(3,7)))
actuals_severity_slimits

array([[     2,      2,     10],
       [    95,   2341,  13995],
       [  6272, 116665, 819971],
       [  1761,  16279, 102233],
       [  1027,   7047,  39759],
       [  7380,  43719, 183831],
       [  2504,  12861,  92199]])

In [6]:
sums2 = df.groupby(['Accident_Severity','Speed_limit']).count().groupby('Accident_Severity').sum()['Accident_Index']
severity_sums = df.groupby(['Accident_Severity','Speed_limit']).count().groupby('Accident_Severity').sum()['Accident_Index'].values
severity_mus = severity_sums/severity_sums.sum()
severity_mus

array([0.01295348, 0.13531997, 0.85172655])

In [7]:
sums = df.groupby(['Speed_limit']).count()['Accident_Index']
slimit_sums = df.groupby(['Speed_limit']).count()['Accident_Index'].values
slimit_sums

array([    14,  16431, 942908, 120273,  47833, 234930, 107564])

In [8]:
expected_by_slimit = np.zeros((7,3))
for i in np.arange(7):
    for j in np.arange(3):
        expected_by_slimit[i][j] = slimit_sums[i] * severity_mus[j]
expected_by_slimit

array([[1.81348655e-01, 1.89447962e+00, 1.19241717e+01],
       [2.12838554e+02, 2.22344247e+03, 1.39947190e+04],
       [1.22139356e+04, 1.27594285e+05, 8.03099780e+05],
       [1.55795334e+03, 1.62753391e+04, 1.02439708e+05],
       [6.19603588e+02, 6.47276026e+03, 4.07406362e+04],
       [3.04315997e+03, 3.17907212e+04, 2.00096119e+05],
       [1.39332763e+03, 1.45555576e+04, 9.16151148e+04]])

In [9]:
#actual number of severity category 1, 2, 3, under speed limits...uh...10,20,30 &c
actuals_severity_slimits

array([[     2,      2,     10],
       [    95,   2341,  13995],
       [  6272, 116665, 819971],
       [  1761,  16279, 102233],
       [  1027,   7047,  39759],
       [  7380,  43719, 183831],
       [  2504,  12861,  92199]])

In [10]:
expected_by_slimit = expected_by_slimit.round().astype(int)
expected_by_slimit
expected_by_slimit.sum(axis=1)

array([    14,  16431, 942908, 120273,  47834, 234930, 107564])

In [12]:
pvalues = np.zeros(7)

# stats.chisquare(actuals_severity_slimits[0],expected_by_slimit[0])[1]

for i in np.arange(7):
    pvalues[i] = stats.chisquare(actuals_severity_slimits[i],expected_by_slimit[i])[1]

pvalues

array([0.00000000e+000, 2.78462053e-016, 0.00000000e+000, 1.46361465e-006,
       6.17893195e-075, 0.00000000e+000, 8.31846765e-237])